# Trabalho 3: Avaliando sistemas de IR

Neste trabalho, o aluno irá explorar o problema de otimização de sistemas de IR através da análise de métricas de avaliação populares para esse tipo de sistema.

Para isso, vamos usar a base CFC (com artigos sobre fibrose cística), a qual encontra-se nesta pasta.

Observação: a idéia aqui é usar o jupyter para mostrar a evolução dos teus experimentos. Então use ele adequadamente. Ou seja, não altere funções do começo para reexecutar algo que foi feito lá embaixo. Neste caso, sobreescreva a função lá embaixo. A leitura do notebook tem que ser feita sequencialmente para ficar fácil, ok?


## Passo 1 - Normalização

Primeira coisa que você vai precisar fazer é ler os arquivos do CFC. Essa é uma parte meio braçal mesmo, mas não tem como escapar dela. Então você terá que criar funções que consigam ler cada arquivo do CFC e parsear eles.


In [37]:
#códigos...
import pandas as pd

FILES = ['cfc/cf74', 'cfc/cf75', 'cfc/cf76', 'cfc/cf77', 'cfc/cf78', 'cfc/cf79']

#for file in FILES:
fp = open('cfc/cf74', 'r', encoding = 'iso-8859-1')
mstrix = []
vector = []
content = fp.readlines()

for line in content:
    
    cod = line[:2]
    
    if (cod.isspace()): 
        pass
    
    elif(cod == 'PN'):
        vector.insert(0,line[3:])
        
    elif(cod == 'RN'):
        vector.insert(1,line[3:])
        
    elif(cod == 'AN'):
        vector.insert(2,line[3:])
        
    elif(cod == 'AU'):
        vector.insert(3,line[3:])       
    
    elif(cod == 'TI'):
        vector.insert(4,line[3:])
        
    elif(cod == 'SO'):
        vector.insert(5,line[3:])
        
    elif(cod == 'MJ'):
        vector.insert(6,line[3:])
        
    elif((cod == 'AB') or (cod == 'EX')):
        vector.insert(7,line[3:])
        
    elif(cod == 'RF'):
        vector.insert(8,line[3:])
        
    elif(cod == 'RF'):
        vector.insert(9,line[3:])
        
    elif(cod == 'CT'):
        vector.insert(10,line[3:])
    else:
        pass
        
fp.close()             
pd.DataFrame(vector)       

,0
0,74168\n
1,00167 \n
2,75010829\n
3,Gotz-M. Ludwig-H. Polymenidis-Z.\n
4,HL-A antigens in cystic fibrosis.\n
5,Z-Kinderheilkd. 1974. 117(3). P 183-6.\n
6,CYSTIC-FIBROSIS: im. HISTOCOMPATIBILITY-ANTIG...
7,HL-A frequencies of 28 patients with cystic fi...
8,001 ASQUITH P LANCET ...
9,74167\n


## Passo 2 - Primeira indexação

Depois de parsear os arquivos, agora é hora de indexar os dados. Ao invés de criar o código do zero, vamos usar o Whoosh, que é uma implementação em python inspirada no Lucene. A documentação pode ser encontrada em https://whoosh.readthedocs.io/en/latest/index.html.

Nesta questão de indexação, temos que fazer algumas escolhas: 
* Como fazer o processo de tokenização? 
* Stemmizar ou não stemmizar, eis a questão...
* Quais campos são úteis para indexação?

Neste momento, você vai tomar suas decisões iniciais. A ideia é você testar, depois, outras soluções para verificar quais tiveram os melhores resultados, entendeu? Então não esqueça de documentar, aqui, qual a sua decisão inicial e depois ir explicando ao longo do notebook os experimentos que está fazendo.

In [ ]:
#códigos...

## Passo 3 - Calculando as métricas de avaliação

Para avaliar a tua decisão, vamos usar como métricas a precisão, recall e f-measure. Faça uma variação da precisão e recall como P@n e R@n, pois serão úteis para os gráficos que serão gerados. 

In [ ]:
#códigos...

## Passo 4 - Gráficos do experimento 1

Apresentar agora os gráficos do teu experimento. Você deve gerar um gráfico PxR, como visto em sala. Para gerar gráficos, você pode usar o matplotlib. Se quiser usar outra biblioteca que ache mais fácil, sem problemas!

In [ ]:
# códigos...

## Passo 5 - Demais experimentos

Agora você irá testar novas configurações de campos e configurações para tentar encontrar qual a que te dá melhores resultados. Mostrar, como um relatório, a evolução do trabalho. Ou seja, mostrar como o trabalho foi evoluindo para você alcançar o melhor resultado (quais modificações foram feitas, como cada modificação influenciou nas métricas, etc).

In [ ]:
# códigos...

## Passo 6 - Quantos resultados eu devo voltar para o usuário?

Um dos problemas de sistemas de IR é determinar quantos resultados você deve retornar ao usuário. Uma forma de entender o comportamento do sistema e qual o corte ideal na lista resultante é através do uso de curvas ROC. Assim, plote a curva ROC do teu sistema e determine, através da análise da curva, qual o ponto ideal de corte para o teu sistema.

In [ ]:
# códigos...